In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import matplotlib.backend_bases
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.stats as sp
from preliminary_tests import conv_jit, main
from sklearn.neighbors import KernelDensity
import numpy as np

In [ ]:
ret = main()

In [ ]:
xs = ret[0].distr()[0]
ps = ret[0].distr()[1]

In [ ]:
X = np.stack([xs, ps]).T
X.shape

In [ ]:
xs[:, np.newaxis]

In [ ]:
X_plot = np.linspace(0,2,1000)[:, np.newaxis]
gaus_dens_est = KernelDensity(kernel="gaussian", bandwidth=.1).fit(xs[:, np.newaxis], sample_weight=ps)
linear_dens_est = KernelDensity(kernel="linear", bandwidth=.1).fit(xs[:, np.newaxis], sample_weight=ps)

In [ ]:
gaus_log_dens = gaus_dens_est.score_samples(X_plot)
linear_log_dens = linear_dens_est.score_samples(X_plot)


In [ ]:
fig, ax = plt.subplots(2,2, sharex=True, sharey=True)
ax[0,0].plot(xs, ps)
ax[1, 0].fill(X_plot[:, 0], np.exp(gaus_log_dens), fc="#AAAAFF")
ax[1, 1].fill(X_plot[:, 0], np.exp(linear_log_dens), fc="#AAAAFF")
ax[1, 0].text(-3.5, 0.31, "Gaussian Kernel Density")
plt.show()